In [ ]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras.models import Model

# path to the model weights files.
weights_path = '../keras/examples/vgg16_weights.h5'
top_model_weights_path = 'bottleneck_fc_model.h5'
# dimensions of our images.
img_width, img_height = 150, 150

#path to the training data git init [project-name]
train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16

# building the VGG16 network
model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(150,150,3))
print('Model loaded.')

# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
top_model.load_weights(top_model_weights_path)

# add the model on top of the convolutional base
#model.add(top_model)
model = Model(input=model.input,output=top_model(model.output))

# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:25]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

# preparing data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

# Generating the training data
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

# Generating the validation data
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

# fine-tune the model
model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    nb_val_samples=nb_validation_samples)

Using TensorFlow backend.


Model loaded.


/home/shubham/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:39: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("se..., inputs=Tensor("in...)`


Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


/home/shubham/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:79: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/home/shubham/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:79: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., validation_data=<keras_pre..., steps_per_epoch=125, epochs=50, validation_steps=800)`


Epoch 1/50
125/125 [==============================] - 1519s 12s/step - loss: 0.4911 - acc: 0.9250 - val_loss: 0.8586 - val_acc: 0.8875
Epoch 2/50
125/125 [==============================] - 1508s 12s/step - loss: 0.5410 - acc: 0.9285 - val_loss: 0.8586 - val_acc: 0.8875
Epoch 3/50
125/125 [==============================] - 1505s 12s/step - loss: 0.4480 - acc: 0.9275 - val_loss: 0.8586 - val_acc: 0.8875
Epoch 4/50
125/125 [==============================] - 1575s 13s/step - loss: 0.5123 - acc: 0.9210 - val_loss: 0.8586 - val_acc: 0.8875
Epoch 5/50
125/125 [==============================] - 1474s 12s/step - loss: 0.5594 - acc: 0.9235 - val_loss: 0.8586 - val_acc: 0.8875
Epoch 6/50
125/125 [==============================] - 1470s 12s/step - loss: 0.5724 - acc: 0.9150 - val_loss: 0.8586 - val_acc: 0.8875
Epoch 7/50
124/125 [============================>.] - ETA: 1s - loss: 0.5003 - acc: 0.9254